# Libs

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay
from utils.futurai_ppd import drop_transitorio_desligado

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed

# Silenciar logs menos importantes do TensorFlow
tf.get_logger().setLevel('ERROR')

import warnings
warnings.filterwarnings('ignore')

# Import Data

In [ ]:
timestamp = 'Timestamp'

df_dataset = pd.read_csv('data/Depurador 762-28-006 - Cozimento.csv', sep=';', decimal='.', encoding='utf-8-sig')
df_dataset.drop(columns=["762P0013.OP", "762F0040.OP", "762F0014.SP", "762H0336.PV", "762H0342.PV", "762N0015.SP", "762P0013.SP", "762-34-073.CR", "762N0015.OP"], inplace=True, errors='ignore')
df_dataset.dropna(inplace=True)
df_dataset[timestamp] = pd.to_datetime(df_dataset[timestamp], format='%Y-%m-%d %H:%M:%S')
df_dataset.head()

# Remove periods off

In [ ]:
pre_process = []
pp_var_ref_desligado = "762-28-006.CR"
pp_valor_ref_desligado = 5
pp_tempo_ref_desligado = 0
pp_pre_corte_transitorio = 0
pp_pos_corte_transitorio = 0
pre_process.append(  
{
   "after_cut": pp_pos_corte_transitorio,
   "interval_off": pp_tempo_ref_desligado,
   "limit_off": pp_valor_ref_desligado,
   "pre_cut": pp_pre_corte_transitorio,
   "variable_off": pp_var_ref_desligado
  })

for pro in pre_process:
    df_dataset,_,_ = drop_transitorio_desligado(df_dataset,pro["variable_off"],pro["limit_off"],pro["interval_off"],timestamp,pre_corte=pro["pre_cut"],pos_corte=pro["after_cut"])
print(f"Dataset shape: {df_dataset.shape}")
df_dataset.head()

# Train with all data

In [ ]:
df_alldata = df_dataset.copy()
eixo_tempo = df_alldata[timestamp]
df_alldata.drop(columns=[timestamp], inplace=True)

scaler = StandardScaler()
scaler.fit(df_alldata)



# Split train test data

In [ ]:
start_date_train = pd.to_datetime('YYYY-mm-dd hh:mm:ss')
end_date_train = pd.to_datetime('YYYY-mm-dd hh:mm:ss')

mask = (df_dataset[timestamp] >= start_date_train) & (df_dataset[timestamp] <= end_date_train)
df_train = df_dataset.loc[mask]
df_train

# Build autoencoder

In [ ]:
# 3. CONSTRUÇÃO DO MODELO AUTOENCODER LSTM
# -----------------------------------------------------------------------------
def create_lstm_autoencoder(input_shape):
    """Cria e compila um modelo de Autoencoder LSTM."""
    # Encoder
    inputs = Input(shape=input_shape)
    # A dimensionalidade latente é 32
    encoded = LSTM(128, activation='relu', return_sequences=False)(inputs)
    encoded = RepeatVector(input_shape[0])(encoded) # Repete o vetor latente para a entrada do decoder
    
    # Decoder
    decoded = LSTM(128, activation='relu', return_sequences=True)(encoded)
    decoded = TimeDistributed(Dense(input_shape[1]))(decoded)
    
    # Autoencoder
    autoencoder = Model(inputs, decoded)
    autoencoder.compile(optimizer='adam', loss='mae') # Mean Absolute Error é robusto a outliers
    
    autoencoder.summary()
    return autoencoder

# Plot and evaluate

In [ ]:
# 4. FUNÇÕES DE PLOTAGEM E AVALIAÇÃO
# -----------------------------------------------------------------------------
def plot_reconstruction_error(error_df, threshold):
    """Plota o erro de reconstrução e o limiar de anomalia."""
    plt.figure(figsize=(15, 6))
    plt.plot(error_df.index, error_df['error'], label='Erro de Reconstrução')
    plt.axhline(y=threshold, color='r', linestyle='--', label='Limiar de Anomalia')
    plt.title('Erro de Reconstrução ao Longo do Tempo')
    plt.xlabel('Data')
    plt.ylabel('Erro (MAE)')
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_anomalies(data, anomaly_indices):
    """Plota a série temporal original destacando as anomalias detectadas."""
    plt.figure(figsize=(15, 8))
    for col in data.columns:
        plt.plot(data.index, data[col], label=col, alpha=0.7)
    
    # Destaca as anomalias detectadas
    anomalies = data.iloc[anomaly_indices]
    for col in anomalies.columns:
        plt.scatter(anomalies.index, anomalies[col], color='red', marker='x', s=50, label=f'Anomalia em {col}')
        
    plt.title('Detecção de Anomalias na Série Temporal Multivariada')
    plt.xlabel('Data')
    plt.ylabel('Valor Normalizado')
    
    # Lidar com legendas duplicadas
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    
    plt.grid(True)
    plt.show()

# Run

In [ ]:
# PARÂMETROS
WINDOW_SIZE = 24 # Usar 24 horas de dados para prever o próximo passo
TRAIN_RATIO = 0.7 # 70% dos dados para treino (período normal)

# 1. Carregar e dividir os dados
df, is_anomaly = generate_data()
n_train = int(len(df) * TRAIN_RATIO)

train_df = df.iloc[:n_train]
test_df = df.iloc[n_train:]
test_labels = is_anomaly.iloc[n_train:]

print(f"Tamanho do dataset de treino: {len(train_df)}")
print(f"Tamanho do dataset de teste: {len(test_df)}")

# 2. Pré-processar e criar janelas
# Treinamos o scaler APENAS com dados de treino para evitar data leakage
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_df)
test_scaled = scaler.transform(test_df)

# Janelas de treino (somente dados normais)
X_train, _ = preprocess_and_window(train_df, WINDOW_SIZE)

# Janelas de teste (inclui anomalias)
X_test, _ = preprocess_and_window(test_df, WINDOW_SIZE)

print(f"Shape das janelas de treino: {X_train.shape}")
print(f"Shape das janelas de teste: {X_test.shape}")

# 3. Criar e treinar o modelo
input_shape = (X_train.shape[1], X_train.shape[2])
autoencoder = create_lstm_autoencoder(input_shape)

history = autoencoder.fit(
    X_train, X_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    shuffle=True,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min')]
)

# 4. Calcular erro de reconstrução
# Erro no treino
train_pred = autoencoder.predict(X_train)
train_mae_loss = np.mean(np.abs(train_pred - X_train), axis=(1, 2))

# Erro no teste
test_pred = autoencoder.predict(X_test)
test_mae_loss = np.mean(np.abs(test_pred - X_test), axis=(1, 2))

# 5. Definir o limiar de anomalia
# Usaremos o 95º percentil do erro de treino como nosso limiar
# Esta é uma abordagem estatística comum.
anomaly_threshold = np.percentile(train_mae_loss, 95)
print(f"\nLimiar de anomalia definido: {anomaly_threshold:.4f}")

# Criar um DataFrame com os erros de teste para visualização
# Os erros correspondem ao final de cada janela
test_error_df = pd.DataFrame(
    index=test_df.index[WINDOW_SIZE-1:],
    data={'error': test_mae_loss}
)

# 6. Visualizar o erro de reconstrução
plot_reconstruction_error(test_error_df, anomaly_threshold)